In [50]:
import gensim
import re

# Загрузка модели
def load_model(model_path):
    print("Загрузка модели...")
    model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=False)
    print("Модель загружена.")
    return model

# Фильтрация существительных
def filter_nouns_from_top(similar_words, topn=100):
    print("\nФильтруем существительные...")
    pat = re.compile(r"(.*)_NOUN")  # Регулярное выражение для поиска существительных
    filtered_words = [(pat.match(word[0]).group(1), word[1]) for word in similar_words if pat.match(word[0])]
    return filtered_words[:topn]

# Метод для получения 100 самых похожих существительных
def get_top_nouns(model, word, topn=100, top_count=1000):
    # Получаем 1000 самых похожих слов
    print(f"\nИщем {top_count} похожих слов для '{word}'...")
    similar_words = model.most_similar(word, topn=top_count)

    # Фильтруем только существительные из полученных слов
    top_nouns = filter_nouns_from_top(similar_words, topn)

    return top_nouns

# Функция поиска похожих слов
def find_similar_words(model, positive, negative, topn=10):
    try:
        similar_words = model.most_similar(positive=positive, negative=negative, topn=topn)
        return similar_words
    except KeyError as e:

        return []

# Функция для поиска линейной комбинации
def find_linear_combination(model, target_words, topn=10):
    # Получаем 100 самых похожих слов для антипротона и калории
    top_nouns_antiproton = get_top_nouns(model, target_words[0], topn=100, top_count=1000)
    top_nouns_caloria = get_top_nouns(model, target_words[1], topn=100, top_count=1000)

    # Выводим найденные существительные для каждого из слов
    print(f"\nНайденные существительные для '{target_words[0]}':")
    for i, (word, similarity) in enumerate(top_nouns_antiproton):
        print(f"{i + 1}: {word} (сходство: {similarity:.4f})")

    print(f"\nНайденные существительные для '{target_words[1]}':")
    for i, (word, similarity) in enumerate(top_nouns_caloria):
        print(f"{i + 1}: {word} (сходство: {similarity:.4f})")

    # Количество всех возможных комбинаций
    total_combinations = len(top_nouns_antiproton) * len(top_nouns_caloria)

    # Перебираем все комбинации слов из этих двух наборов
    current_combination = 0  # Для отслеживания прогресса

    for word1, _ in top_nouns_antiproton:
        for word2, _ in top_nouns_caloria:
            # Проверяем линейную комбинацию существительных
            positive = [word1 + "_NOUN", word2 + "_NOUN"]  # Слова для сложения
            negative = []  # Слова для вычитания

            # Найти похожие слова
            similar_words = model.most_similar(positive=positive, negative=negative, topn=topn)

            # Проверяем, попадут ли слова «антипротон» и «калория» в топ-10
            similar_word_list = [word[0] for word in similar_words]
            if target_words[0] in similar_word_list and target_words[1] in similar_word_list:
                print(f"\nКомбинация слов '{word1}' и '{word2}' привела к результату:")
                for i, (word, similarity) in enumerate(similar_words):
                    print(f"{i + 1}: {word} (сходство: {similarity:.4f})")
                return  # Завершаем выполнение, если нашли комбинацию

            # Обновляем прогресс
            current_combination += 1
            progress = (current_combination / total_combinations) * 100  # Рассчитываем процент
            print(f"\rПрогресс: {progress:.2f}%", end="")  # Выводим прогресс в одну строку

    print("\nНе удалось найти подходящую линейную комбинацию существительных.")


In [51]:
# Путь к предобученной модели
model_path = "/content/drive/MyDrive/Colab Notebooks/cbow.txt"  # Укажите путь к вашей модели

# Загрузить модель
word2vec = load_model(model_path)

Загрузка модели...
Модель загружена.


In [63]:
# Целевые слова
target_words = ["антипротон_NOUN", "калория_NOUN"]

# Найти линейную комбинацию слов, которая приведет к целевым словам в первой десятке
find_linear_combination(word2vec, target_words, topn=10)



Ищем 1000 похожих слов для 'антипротон_NOUN'...

Фильтруем существительные...

Ищем 1000 похожих слов для 'калория_NOUN'...

Фильтруем существительные...

Найденные существительные для 'антипротон_NOUN':
1: антиводород (сходство: 0.7341)
2: нейтрино (сходство: 0.7021)
3: пи-мезон (сходство: 0.6828)
4: протон (сходство: 0.6782)
5: электрон (сходство: 0.6666)
6: позитрон (сходство: 0.6577)
7: фотон (сходство: 0.6477)
8: античастица (сходство: 0.6469)
9: мезон (сходство: 0.6422)
10: кварк (сходство: 0.6375)
11: мю-мезон (сходство: 0.6352)
12: ускоритель (сходство: 0.6245)
13: антивещество (сходство: 0.6239)
14: нейтрон (сходство: 0.6217)
15: аннигиляция (сходство: 0.6057)
16: электрона (сходство: 0.6047)
17: квант (сходство: 0.6033)
18: мезона (сходство: 0.5916)
19: частица (сходство: 0.5862)
20: ксенон (сходство: 0.5787)
21: -частица (сходство: 0.5755)
22: дейтерий (сходство: 0.5721)
23: плазм (сходство: 0.5700)
24: античастиц (сходство: 0.5692)
25: адрон (сходство: 0.5642)
26: атом (сх